In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../raw_data/7.범죄검거원표.csv")
df

,date,jur_stn,crm
0,201701,경남마산중부경찰서,사기
1,201701,경남마산중부경찰서,경범죄처벌법위반
2,201612,경남마산중부경찰서,도로교통법위반(음주운전)
3,201702,경남마산중부경찰서,위계공무집행방해
4,201702,경남마산중부경찰서,위계공무집행방해
...,...,...,...
928560,201912,경남창원중부경찰서,교통사고처리특례법위반(치상)
928561,201912,경남창원중부경찰서,사기
928562,201912,경남창원중부경찰서,사기
928563,201912,경남창원중부경찰서,사기


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928565 entries, 0 to 928564
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     928565 non-null  int64 
 1   jur_stn  928565 non-null  object
 2   crm      928565 non-null  object
dtypes: int64(1), object(2)
memory usage: 21.3+ MB


## 1. 연도별 정리

In [6]:
## 연도별 정리를 위해 날짜  str변환
df["date"] = df["date"].astype(str)

In [8]:
# 17, 18, 19데이터만 추출
df = df[df["date"].str.contains("2017|2018|2019")].reset_index(drop=True)
df

,date,jur_stn,crm
0,201701,경남마산중부경찰서,사기
1,201701,경남마산중부경찰서,경범죄처벌법위반
2,201702,경남마산중부경찰서,위계공무집행방해
3,201702,경남마산중부경찰서,위계공무집행방해
4,201702,경남마산중부경찰서,도로교통법위반(무면허운전)
...,...,...,...
928305,201912,경남창원중부경찰서,교통사고처리특례법위반(치상)
928306,201912,경남창원중부경찰서,사기
928307,201912,경남창원중부경찰서,사기
928308,201912,경남창원중부경찰서,사기


In [17]:
# 연도만 가져오기
df.loc[df[df["date"].str.contains("2017")]["date"].index,"date"]="2017"
df.loc[df[df["date"].str.contains("2018")]["date"].index,"date"]="2018"
df.loc[df[df["date"].str.contains("2019")]["date"].index,"date"]="2019"

In [20]:
df

,date,jur_stn,crm
0,2017,경남마산중부경찰서,사기
1,2017,경남마산중부경찰서,경범죄처벌법위반
2,2017,경남마산중부경찰서,위계공무집행방해
3,2017,경남마산중부경찰서,위계공무집행방해
4,2017,경남마산중부경찰서,도로교통법위반(무면허운전)
...,...,...,...
928305,2019,경남창원중부경찰서,교통사고처리특례법위반(치상)
928306,2019,경남창원중부경찰서,사기
928307,2019,경남창원중부경찰서,사기
928308,2019,경남창원중부경찰서,사기


## 2. 범죄명 별 처리

+ 문1 : 절도 , 폭력
+ 문2 : 강도 , 살인
+ 문3 : 교통사고
+ 문4 : 기초질서, 집회시위질서 등 법질서 준수도
+ 문5 : 전반적 안전도(all)

In [21]:
절폭_list = ['125260110', '125260200', '139100110', '206059800', '206060900',
       '209014300', '간음약취', '간음약취미수', '간음유인', '감금', '감금미수', '감금치상',
       '경계침범', '공갈', '공갈미수', '공익건조물파괴', '공익건조물파괴미수', '문서손괴', '문서손괴교사',
       '문서은닉', '미성년자약취', '미성년자약취미수', '미성년자약취예비', '미성년자유인', '미성년자유인미수',
       '상습공갈', '상습공갈미수', '상습상해', '상습야간건조물침입절도', '상습야간건조물침입절도미수',
       '상습야간주거침입절도', '상습야간주거침입절도미수', '상습자동차불법사용', '상습절도', '상습절도미수',
       '상습존속상해', '상습존속폭행', '상습특수상해', '상습특수상해미수', '상습특수절도', '상습특수절도미수',
       '상습특수존속협박', '상습특수폭행', '상습폭행', '상습협박', '상해', '상해교사', '상해미수', '상해방조',
       '상해치사', '성매매약취', '성매매유인', '아동학대범죄의처벌등에관한특례법위반(상습폭행)', '야간건조물침입절도',
       '야간건조물침입절도미수', '야간방실침입절도', '야간방실침입절도교사', '야간방실침입절도미수', '야간주거침입절도',
       '야간주거침입절도미수', '야간주거침입절도방조', '영리약취미수', '영리약취예비', '영리유인',
       '원동기장치자전거불법사용', '자동차불법사용', '자동차불법사용미수', '재물손괴', '재물손괴교사', '재물손괴미수',
       '재물손괴치상', '재물은닉', '재물은닉미수', '재물은닉방조', '전자기록등손괴', '절도', '절도교사',
       '절도미수', '절도방조', '존속감금', '존속상해', '존속상해치사', '존속체포', '존속폭행', '존속폭행치사',
       '존속폭행치상', '존속협박', '중감금', '중감금치상', '중상해', '중손괴', '중체포치상', '체포',
       '체포미수', '추행약취', '추행약취미수', '추행유인', '특수감금', '특수감금치상', '특수공갈',
       '특수공갈미수', '특수상해', '특수상해미수', '특수재물손괴', '특수재물손괴교사', '특수재물손괴미수',
       '특수전자기록등손괴', '특수절도', '특수절도교사', '특수절도미수', '특수절도방조', '특수존속감금',
       '특수존속상해', '특수존속폭행', '특수존속협박', '특수존속협박미수', '특수중감금', '특수중감금치상',
       '특수중상해', '특수체포미수', '특수폭행', '특수폭행교사', '특수폭행치사', '특수폭행치상', '특수협박',
       '특수협박미수', '특정경제범죄가중처벌등에관한법률위반(공갈)', '특정범죄가중처벌등에관한법률위반(13세미만약취·유인)',
       '특정범죄가중처벌등에관한법률위반(보복범죄등)', '특정범죄가중처벌등에관한법률위반(보복상해등)',
       '특정범죄가중처벌등에관한법률위반(보복폭행등)', '특정범죄가중처벌등에관한법률위반(보복협박등)',
       '특정범죄가중처벌등에관한법률위반(운전자폭행)', '특정범죄가중처벌등에관한법률위반(운전자폭행등)',
       '특정범죄가중처벌등에관한법률위반(절도)', '특정범죄가중처벌등에관한법률위반(절도)방조',
       '폭력행위등처벌에관한법률위반(공동감금)', '폭력행위등처벌에관한법률위반(공동강요)',
       '폭력행위등처벌에관한법률위반(공동건조물침입)', '폭력행위등처벌에관한법률위반(공동공갈)',
       '폭력행위등처벌에관한법률위반(공동공갈미수)', '폭력행위등처벌에관한법률위반(공동상해)',
       '폭력행위등처벌에관한법률위반(공동재물손괴등)', '폭력행위등처벌에관한법률위반(공동존속상해)',
       '폭력행위등처벌에관한법률위반(공동존속폭행)', '폭력행위등처벌에관한법률위반(공동존속협박)',
       '폭력행위등처벌에관한법률위반(공동주거침입)', '폭력행위등처벌에관한법률위반(공동체포)',
       '폭력행위등처벌에관한법률위반(공동퇴거불응)', '폭력행위등처벌에관한법률위반(공동폭행)',
       '폭력행위등처벌에관한법률위반(공동폭행)교사', '폭력행위등처벌에관한법률위반(공동협박)',
       '폭력행위등처벌에관한법률위반(단체등의공동강요)', '폭력행위등처벌에관한법률위반(단체등의공동상해)',
       '폭력행위등처벌에관한법률위반(단체등의공동재물손괴등)', '폭력행위등처벌에관한법률위반(단체등의공동주거침입)',
       '폭력행위등처벌에관한법률위반(단체등의공동퇴거불응)', '폭력행위등처벌에관한법률위반(단체등의공동폭행)',
       '폭력행위등처벌에관한법률위반(단체등의공동협박)', '폭력행위등처벌에관한법률위반(단체등의공용물건손상)',
       '폭력행위등처벌에관한법률위반(단체등의구성·활동)', '폭력행위등처벌에관한법률위반(단체등의상습집단·흉기등상해)',
       '폭력행위등처벌에관한법률위반(단체등의이용·지원)', '폭력행위등처벌에관한법률위반(단체등의집단·흉기등상해)',
       '폭력행위등처벌에관한법률위반(단체등의집단·흉기등폭행)', '폭력행위등처벌에관한법률위반(상습공갈)',
       '폭력행위등처벌에관한법률위반(상습상해)', '폭력행위등처벌에관한법률위반(상습재물손괴등)',
       '폭력행위등처벌에관한법률위반(상습집단·흉기등상해)', '폭력행위등처벌에관한법률위반(상습집단·흉기등폭행)',
       '폭력행위등처벌에관한법률위반(상습특수상해)', '폭력행위등처벌에관한법률위반(상습특수협박)',
       '폭력행위등처벌에관한법률위반(상습폭행)', '폭력행위등처벌에관한법률위반(상습협박)',
       '폭력행위등처벌에관한법률위반(우범자)', '폭력행위등처벌에관한법률위반(집단·흉기등공갈)',
       '폭력행위등처벌에관한법률위반(집단·흉기등상해)', '폭력행위등처벌에관한법률위반(집단·흉기등주거침입)',
       '폭력행위등처벌에관한법률위반(집단·흉기등퇴거불응)', '폭력행위등처벌에관한법률위반(집단·흉기등폭행)',
       '폭력행위등처벌에관한법률위반(집단·흉기등협박)', '폭행', '폭행교사', '폭행방조', '폭행치사', '폭행치상',
       '피약취자상해', '협박', '협박교사', '협박미수', '협박방조']

In [22]:
강살_list = ['강도', '강도강간', '강도강간미수', '강도미수', '강도살인', '강도살인미수', '강도상해', '강도상해미수',
       '강도예비', '강도음모', '강도치상', '살인', '살인교사', '살인미수', '살인방조', '살인예비',
       '살인음모', '영아살해', '인질강도', '인질강도미수', '자살교사미수', '자살방조', '자살방조미수',
       '존속살해', '존속살해교사', '존속살해미수', '존속살해예비', '준강도', '준강도미수', '촉탁살인',
       '특수강도', '특수강도미수', '특수강도예비', '특정범죄가중처벌등에관한법률위반(강도)']

In [23]:
교통_list = ['과실일반교통방해', '교통사고처리특례법위반', '교통사고처리특례법위반(치사)', '교통사고처리특례법위반(치사)교사',
       '교통사고처리특례법위반(치사)방조)', '교통사고처리특례법위반(치상)', '교통사고처리특례법위반(치상)교사',
       '교통사고처리특례법위반(치상)방조', '기차교통방해', '도로교통법위반', '도로교통법위반(공동위험행위)',
       '도로교통법위반(무면허운전)', '도로교통법위반(무면허운전)교사', '도로교통법위반(무면허운전)방조',
       '도로교통법위반(사고후미조치)', '도로교통법위반(사고후미조치)방조', '도로교통법위반(음주운전)',
       '도로교통법위반(음주운전)교사', '도로교통법위반(음주운전)방조', '도로교통법위반(음주측정거부)',
       '도로교통법위반(음주측정거부)교사', '도로교통법위반(음주측정거부)방조', '도로법위반', '도시교통정비촉진법',
       '업무상과실일반교통방해', '일반교통방해', '일반교통방해치상', '자동차교통방해',
       '특정범죄가중처벌등에관한법률위반(도주차량)']

In [24]:
질서_list = ['119구조·구급에관한법률위반', '137070100', '137070110', '205010500',
       '207010300', '209014400', '209014500', '209014800', '209014900',
       '209015000', '209015100', '209015101', '209015102', '209015200',
       '209926200', '210020700', '212030300', '214030400', '216010100',
       '229030400', '229030500', '231044100', '234020700', '234021000',
       '235040400', '236010400', '236020400', '236020700', '236020800',
       '239022100', '242080500', '가맹사업거래의공정화에관한법률위반', '가사소송법위반', '가스공급방해',
       '가스방출', '가정폭력방지및피해자보호등에관한법률위반', '가정폭력범죄의처벌등에관한특례법위반',
       '가족관계의등록등에관한법률위반', '가축및축산물이력관리에관한법률위반', '가축분뇨의관리및이용에관한법률위반',
       '가축전염병예방법위반', '가혹행위', '감염병의예방및관리에관한법률위반', '감정평가및감정평가사에관한법률위반',
       '강요', '강요교사', '강요미수', '강제집행면탈', '개발제한구역의지정및관리에관한특별조치법위반',
       '개인정보보호법위반', '건강기능식품에관한법률위반', '건설기계관리법위반', '건설기술관리법위반',
       '건설기술진흥법위반', '건설산업기본법위반', '건설업법위반', '건설폐기물의재활용촉진에관한법률위반', '건조물수색',
       '건조물침입', '건조물침입교사', '건조물침입미수', '건조물퇴거불응', '건축물의분양에관한법률위반', '건축법',
       '건축법위반', '건축사법위반', '게임산업진흥에관한법률위반', '게임산업진흥에관한법률위반방조', '결핵예방법위반',
       '결혼중개업의관리에관한법률위반', '경륜경정법위반', '경매방해', '경매방해방조', '경범죄처벌법위반',
       '경비업법위반', '경찰제복및경찰장비의규제에관한법률위반', '계량에관한법률위반', '고등교육법위반',
       '고압가스안전관리법위반', '고용보험법위반', '골재채취법위반', '공간정보의구축및관리등에관한법률위반',
       '공공감사에관한법률위반', '공공기록물관리에관한법률위반', '공공단체등위탁선거에관한법률위반',
       '공공주택건설등에관한특별법위반', '공공주택특별법위반', '공기호부정사용', '공기호위조', '공동주택관리법위반',
       '공무상보관물무효', '공무상보관물은닉', '공무상봉인무효', '공무상비밀누설', '공무상비밀문서개봉',
       '공무상표시무효', '공무상표시손상', '공무상표시은닉', '공무원자격사칭', '공무집행방해', '공문서변조',
       '공문서변조교사', '공문서부정행사', '공문서부정행사미수', '공문서부정행사방조', '공문서위조', '공문서위조교사',
       '공문서위조방조', '공연법위반', '공연음란', '공용건조물방화', '공용건조물방화미수', '공용건조물방화예비',
       '공용건조물파괴', '공용물건무효', '공용물건손상', '공용물건손상미수', '공용물건은닉', '공용서류무효',
       '공용서류무효미수', '공용서류손상', '공용전자기록등손상', '공유수면관리및매립에관한법률위반',
       '공유재산및물품관리법위반', '공익건조물방화미수', '공익법인의설립운영에관한법률위반',
       '공익사업을위한토지등의취득및보상에관한법률위반', '공익신고자보호법위반', '공인노무사법위반', '공인부정사용',
       '공인위조', '공인중개사법위반', '공인중개사의업무및부동산거래신고에관한법률', '공인회계사법위반',
       '공장및광업재단저당법위반', '공전자기록등변작', '공전자기록등불실기재', '공전자기록등위작', '공정증서원본불실기재',
       '공중위생관리법위반', '공중위생법위반', '공증인법위반', '공직선거법위반', '공직자윤리법위반', '과실가스방출',
       '과실치사', '과실치상', '과실폭발물파열', '과실폭발성물건파열', '관광진흥법위반', '관세법위반',
       '교육환경보호에관한법률위반', '국가공무원법위반', '국가기술자격법위반', '국가보안법위반',
       '국가보안법위반(잠입·탈출)', '국가보안법위반(찬양·고무등)', '국가보안법위반(회합·통신등)', '국기모독',
       '국민건강보험법위반', '국민건강증진법위반', '국민기초생활보장법위반', '국민연금법위반', '국민체육진흥법',
       '국민체육진흥법위반', '국민체육진흥법위반(도박개장등)', '국민체육진흥법위반(도박개장등)방조',
       '국민체육진흥법위반(도박등)', '국유재산법위반', '국제상거래에있어서외국공무원에대한뇌물방지법위반',
       '국토의계획및이용에관한법률위반', '국회에서의증언·감정등에관한법률위반', '군무이탈',
       '군복및군용장구의단속에관한법률위반', '군사기지및군사시설보호법위반', '군용물등범죄에관한특별조치법위반', '군용물절도',
       '군인등강제추행', '군형법위반', '권리행사방해', '근로기준법위반', '근로자직업능력개발법위반',
       '금융실명거래및비밀보장에관한법률위반', '금융실명거래및비밀보장에관한법률위반방조', '기부금품의모집및사용에관한법률위반',
       '기초연금법위반', '기타위반', '낙태', '낙태교사', '낙태방조', '낚시관리및육성법위반',
       '남녀고용평등과일·가정양립지원에관한법률위반', '남북교류협력에관한법률위반', '내란선동', '내란선전', '내란실행',
       '내란예비', '내수면어업법위반', '노동위원회법위반', '노인복지법위반', '노인장기요양보험법위반',
       '농수산물유통및가격안정에관한법률위반', '농수산물의원산지표시에관한법률위반', '농약관리법위반', '농어촌정비법위반',
       '농업협동조합법위반', '농지법위반', '뇌물공여', '뇌물공여의사표시', '뇌물수수', '담배사업법위반',
       '대기환경보전법위반', '대리점거래의공정화에관한법률위반', '대부업등의등록및금융이용자보호에관한법률위반',
       '대부업등의등록및금융이용자보호에관한법률위반방조', '대부업의등록및금융이용자보호에관한법률위반', '대외무역법위반',
       '대중문화예술산업발전법위반', '도박', '도박개장', '도박개장방조', '도박공간개설', '도박공간개설방조',
       '도박방조', '도박장소개설', '도시가스사업법위반', '도시개발법위반', '도시공원및녹지등에관한법률위반',
       '도시및주거환경정비법위반', '도주', '독점규제및공정거래에관한법률위반', '독직가혹행위', '독직폭행',
       '동물보호법위반', '동물원및수족관의관리에관한법률위반', '디자인보호법위반', '라디오에의한명예훼손',
       '마약류관리에관한법률위반(대마)', '마약류관리에관한법률위반(마약)', '마약류관리에관한법률위반(향정)',
       '마약류관리에관한법률위반(향정)교사', '마약류관리에관한법률위반(향정)방조', '마약류불법거래방지에관한특례법위반',
       '매장문화재보호및조사에관한법률위반', '먹는물관리법위반', '면허증불실기재', '면허증불실기재미수', '명예훼손',
       '명예훼손교사', '모욕', '모욕교사', '모자보건법위반', '모해위증', '모해위증교사', '모해증거변조',
       '모해증거위조', '모해증거인멸', '모해증인은닉', '목재의지속가능한이용에관한법률위반', '무고', '무고교사',
       '무단이탈', '문서개봉', '문화재보호법위반', '문화재수리등에관한법률위반', '물류시설의개발및운영에관한법률위반',
       '물환경보전법위반', '미상', '민간임대주택에관한특별법위반', '민사소송법', '민사집행법위반',
       '방문판매등에관한법률위반', '방실수색', '방실침입', '방실침입미수', '방위사업법위반', '방화연소', '배임',
       '배임미수', '배임방조', '배임수재', '배임수재미수', '배임증재', '범인도피', '범인도피교사', '범인은닉',
       '범죄단체가입', '범죄단체조직', '범죄단체활동', '범죄수익은닉의규제및처벌등에관한법률위반', '법무사법위반',
       '변리사법위반', '변조공문서행사', '변조사문서행사', '변호사법위반', '변호사법위반방조', '병역법위반',
       '보건범죄단속에관한특별조치법위반(부정식품제조등)', '보건범죄단속에관한특별조치법위반(부정의료업자)',
       '보건범죄단속에관한특별조치법위반(부정의약품제조등)', '보조금관리법위반', '보조금관리에관한법률위반',
       '보조금관리에관한법률위반교사', '보조금관리에관한법률위반방조', '보험사기방지특별법위반', '보험사기방지특별법위반방조',
       '보험업법위반', '보험업법위반방조', '보호관찰등에관한법률위반', '복권및복권기금법위반', '복표발매', '부당이득',
       '부동산강제집행효용침해', '부동산개발업의관리및육성에관한법률위반', '부동산거래신고등에관한법률위반',
       '부동산등기특별조치법위반', '부동산실권리자명의등기에관한법률위반', '부동산중개업법위반',
       '부정경쟁방지및영업비밀보호에관한법률', '부정경쟁방지및영업비밀보호에관한법률위반',
       '부정경쟁방지및영업비밀보호에관한법률위반(영업비밀국외누설등)', '부정경쟁방지및영업비밀보호에관한법률위반(영업비밀누설등)',
       '부정사용공기호행사', '부정수표단속법위반', '부정청탁및금품등수수의금지에관한법률위반',
       '부패방지및국민권익위원회의설치와운영에관한법률위반', '북한이탈주민의보호및정착지원에관한법률위반', '분묘발굴',
       '불공정무역행위조사및산업피해구제에관한법률위반', '불실기재공정증서원본행사', '불실기재면허증행사', '비료관리법위반',
       '비밀침해', '비영리민간단체지원법위반', '빈집및소규모주택정비에관한특례법위반', '사격및사격장안전관리에관한법률위반',
       '사기', '사기교사', '사기미수', '사기미수방조', '사기방조', '사도법위반', '사료관리법위반',
       '사립학교법위반', '사문서변조', '사문서부정행사', '사문서위조', '사문서위조교사', '사문서위조미수',
       '사서명부정사용', '사서명위조', '사인부정사용', '사인위조', '사자명예훼손', '사전뇌물수수',
       '사전자기록등변작', '사전자기록등위작', '사체유기', '사행행위등규제및처벌특례법위반', '사행행위등규제법위반',
       '사회복지사업법위반', '사회서비스이용및이용권관리에관한법률위반', '산림보호법위반',
       '산림자원의조성및관리에관한법률위반', '산업기술의유출방지및보호에관한법률위반', '산업안전보건법위반',
       '산업재해보상보험법위반', '산업집적활성화및공장설립에관한법률위반', '산업표준화법위반', '산지관리법위반',
       '상관모욕', '상법위반', '상습도박', '상습사기', '상습사기미수', '상습장물운반', '상습장물취득',
       '상표법위반', '상훈법위반', '새마을금고법위반', '생명윤리및안전에관한법률위반',
       '생물다양성보전및이용에관한법률위반', '석면안전관리법위반', '석유및석유대체연료사업법위반', '선박안전법위반',
       '선박침입', '설교방해', '성매매방지및피해자보호등에관한법률위반', '성매매알선등행위의처벌에관한법률위반',
       '성매매알선등행위의처벌에관한법률위반(성매매)', '성매매알선등행위의처벌에관한법률위반(성매매)방조',
       '성매매알선등행위의처벌에관한법률위반(성매매강요등)', '성매매알선등행위의처벌에관한법률위반(성매매광고)',
       '성매매알선등행위의처벌에관한법률위반(성매매광고)교사', '성매매알선등행위의처벌에관한법률위반(성매매광고)방조',
       '성매매알선등행위의처벌에관한법률위반(성매매알선등)', '성매매알선등행위의처벌에관한법률위반(성매매알선등)교사',
       '성매매알선등행위의처벌에관한법률위반(성매매알선등)방조', '성매매알선등행위의처벌에관한법률위반(아동·청소년)',
       '성폭력방지및피해자보호등에관한법률위반(비밀엄수)', '성폭력범죄의처벌등에관한특례법위반',
       '성폭력범죄의처벌등에관한특례법위반(비밀준수등)', '성폭력범죄의처벌등에관한특례법위반(성적목적공공장소침입)',
       '성폭력범죄의처벌등에관한특례법위반(성적목적다중이용장소침입)', '성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영)',
       '성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영)미수', '성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영)방조',
       '성폭력범죄의처벌등에관한특례법위반(통신매체이용음란)', '성폭력범죄의처벌및피해자보호등에관한법률위반(비밀엄수)',
       '성폭력범죄의처벌및피해자보호등에관한법률위반(카메라등이용촬영)',
       '성폭력범죄의처벌및피해자보호등에관한법률위반(카메라등이용촬영)미수',
       '성폭력범죄의처벌및피해자보호등에관한법률위반(통신매체이용음란)', '성폭력범죄의피해자보호등에관한법률위반(비밀엄수)',
       '세무사법위반', '세월호선체조사위원회의설치및운영에관한특별법위반', '소나무재선충병방제특별법위반', '소방기본법위반',
       '소방시설공사업법위반', '소방시설설치유지및안전관리에관한법률위반', '소비자생활협동조합법위반', '소음·진동관리법위반',
       '소하천정비법위반', '수도권대기환경개선에관한특별법위반', '수도법위반', '수도불통', '수뢰후부정처사',
       '수산업법위반', '수산직접지불제시행에관한법률위반', '수상레저안전법위반', '수의사법위반',
       '수입식품안전관리특별법위반', '수질및수생태계보전에관한법률위반', '승강기시설안전관리법위반', '식물신품종보호법위반',
       '식품위생법위반', '신용정보의이용및보호에관한법률위반', '신용협동조합법위반', '신용훼손', '신체수색',
       '실용신안법위반', '실종아동등의보호및지원에관한법률위반', '실화', '아동·청소년의성보호에관한법률위반',
       '아동·청소년의성보호에관한법률위반(강요행위등)', '아동·청소년의성보호에관한법률위반(매매)',
       '아동·청소년의성보호에관한법률위반(매매)교사', '아동·청소년의성보호에관한법률위반(성매수등)',
       '아동·청소년의성보호에관한법률위반(알선영업행위등)', '아동·청소년의성보호에관한법률위반(음란물소지)',
       '아동·청소년의성보호에관한법률위반(음란물온라인서비스제공)', '아동·청소년의성보호에관한법률위반(음란물제작·배포등)',
       '아동복지법위반', '아동복지법위반(상습아동에대한음행강요·매개·성희롱등)', '아동복지법위반(상습아동유기·방임)',
       '아동복지법위반(상습아동학대)', '아동복지법위반(아동에대한성희롱등)', '아동복지법위반(아동에대한음행강요)',
       '아동복지법위반(아동에대한음행강요·매개·성희롱등)', '아동복지법위반(아동유기·방임)',
       '아동복지법위반(아동유기·방임)방조', '아동복지법위반(아동학대)', '아동학대범죄의처벌등에관한특례법위반',
       '아동학대범죄의처벌등에관한특례법위반(보도금지의무위반)', '아동학대범죄의처벌등에관한특례법위반(보호처분등의불이행)',
       '아동학대범죄의처벌등에관한특례법위반(비밀엄수의무위반)', '아동학대범죄의처벌등에관한특례법위반(상습학대)',
       '아동학대범죄의처벌등에관한특례법위반(아동복지시설종사등의아동학대)교사',
       '아동학대범죄의처벌등에관한특례법위반(아동복지시설종사자등의아동학대)',
       '아동학대범죄의처벌등에관한특례법위반(아동복지시설종사자등의아동학대가중처벌)',
       '아동학대범죄의처벌등에관한특례법위반(아동복지시설종사자등의아동학대가중처벌)교사',
       '아동학대범죄의처벌등에관한특례법위반(아동학대중상해)', '아동학대범죄의처벌등에관한특례법위반(아동학대치사)',
       '아동학대범죄의처벌등에관한특례법위반(이수명령불이행)', '악취방지법위반', '액화석유가스의안전관리및사업법',
       '액화석유가스의안전관리및사업법위반', '야생동·식물보호법위반', '야생생물보호및관리에관한법률위반', '약사법위반',
       '어린이놀이시설안전관리법위반', '어린이제품안전특별법위반', '어선법위반', '업무방해', '업무방해교사',
       '업무방해방조', '업무상과실자동차전복', '업무상과실장물보관', '업무상과실장물취득', '업무상과실전기공급방해',
       '업무상과실치사', '업무상과실치상', '업무상과실폭발성물건파열', '업무상배임', '업무상배임미수',
       '업무상비밀누설', '업무상승낙낙태치사', '업무상실화', '업무상횡령', '업무상횡령미수', '에너지이용합리화법위반',
       '여객자동차운수사업법위반', '여권법위반', '여신전문금융업법위반', '여신전문금융업법위반미수', '영아유기',
       '영아유기치사', '영유아보육법위반', '영화및비디오물의진흥에관한법률위반', '예배방해', '예비군법위반',
       '옥외광고물등관리법위반', '옥외광고물등의관리와옥외광고산업진흥에관한법률위반', '외국사절폭행', '외국사절협박',
       '외국인근로자의고용등에관한법률', '외국인근로자의고용등에관한법률위반', '외국인근로자의고용등에관한법률위반교사',
       '외국인토지법위반', '외국통화위조', '외국환거래법위반', '외국환관리법위반', '우편법위반', '우표소인말소',
       '위계공무집행방해', '위계공무집행방해교사', '위생용품관리법위반', '위조공기호행사', '위조공문서행사',
       '위조사문서행사', '위조외국통화지정행사', '위조외국통화행사', '위조외국통화행사미수', '위조유가증권행사',
       '위조유가증권행사미수', '위조통화지정행사', '위조통화지정행사방조', '위조통화취득', '위조통화행사', '위증',
       '위증교사', '위증방조', '위치정보의보호및이용등에관한법률위반', '위험물안전관리법위반', '유가증권변조',
       '유가증권위조', '유골유기', '유기', '유기치사', '유사수신행위의규제에관한법률위반',
       '유사수신행위의규제에관한법률위반방조', '유아교육법위반', '유전자변형생물체의국가간이동등에관한법률위반',
       '유통산업발전법위반', '유해화학물질관리법위반', '유해화학물질관리법위반(환각물질흡입)', '음란물건반포',
       '음반·비디오물및게임물에관한법률위반', '음악산업진흥에관한법률위반', '음화반포', '음화전시', '음화제조',
       '음화판매', '응급의료에관한법률위반', '의료급여법위반', '의료기기법위반', '의료기사등에관한법률위반',
       '의료기사법위반', '의료법위반', '의료법위반교사', '의료법위반방조', '의료보험법위반',
       '의료해외진출및외국인환자유치지원에관한법률위반', '의무경찰대성치및운영에관한법률위반', '이자제한법위반', '인질강요',
       '일반건조물방화', '일반건조물방화미수', '일반건조물방화예비', '일반물건방화', '일반물건방화미수',
       '일반자동차방화', '일반자동차방화미수', '일반자동차방화예비', '임대주택법위반', '임업및산촌진흥촉진에관한법률위반',
       '입찰방해', '자격기본법위반', '자격모용공문서작성', '자격모용사문서작성', '자격모용유가증권작성',
       '자기소유일반물건방화', '자기소유자동차방화', '자동차관리법위반', '자동차손해배상보장법위반', '자동차수색',
       '자동차수색미수', '자본시장과금융투자업에관한법률위반', '자연공원법위반', '장례식방해', '장물보관', '장물알선',
       '장물양도', '장물운반', '장물운반교사', '장물취득', '장사등에관한법률위반', '장사등에관한법률위반교사',
       '장애인·노인·임산부등의편의증진보장에관한법률위반', '장애인고용촉진및직업재활법위반', '장애인등에대한특수교육법위반',
       '장애인복지법위반', '장애인차별금지및권리구제등에관한법률위반', '장애인활동지원에관한법률위반',
       '재난및안전관리기본법위반', '저작권법위반', '저작권법위반교사', '저작권법위반방조', '전기공급방해',
       '전기공사업법위반', '전기사업법위반', '전기용품및생활용품안전관리법위반', '전기용품안전관리법위반',
       '전기통신금융사기피해금환급에관한특별법위반', '전기통신금융사기피해금환급에관한특별법위반방조',
       '전기통신금융사기피해방지및피해금환급에관한특별법위반', '전기통신금융사기피해방지및피해금환급에관한특별법위반방조',
       '전기통신기본법위반', '전기통신사업법위반', '전자거래기본법위반', '전자금융거래법위반', '전자금융거래법위반교사',
       '전자기록등내용탐지', '전자기록등손괴업무방해', '전자상거래등에서의소비자보호에관한법률',
       '전자상거래등에서의소비자보호에관한법률위반', '전자서명법위반', '전자정부법위반', '전파법위반', '점유강취',
       '점유이탈군용물횡령', '점유이탈물횡령', '정당법위반', '정보통신공사업법위반', '정보통신기반보호법위반',
       '정보통신망이용촉진등에관한법률위반', '정보통신망이용촉진및정보보호등에관한법률위반',
       '정보통신망이용촉진및정보보호등에관한법률위반(개인정보누설등)', '정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)',
       '정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)교사', '정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)방조',
       '정보통신망이용촉진및정보보호등에관한법률위반(음란물유포)', '정보통신망이용촉진및정보보호등에관한법률위반(음란물유포)방조',
       '정보통신망이용촉진및정보보호등에관한법률위반(정보통신망침해등)',
       '정보통신망이용촉진및정보보호등에관한법률위반(정보통신망침해등)교사',
       '정보통신망이용촉진및정보보호등에관한법률위반(정보통신망침해등)방조', '정신건강증진및정신질환자복지서비스지원에관한법률위반',
       '정신보건법위반', '정치자금법위반', '제3자뇌물교부', '제3자뇌물수수', '제3자뇌물취득',
       '제대혈관리및연구에관한법률위반', '제사방해', '제주특별자치도설치및국제자유도시조성을위한특별법위반',
       '제품안전기본법위반', '조세범처벌법위반', '존속유기', '존속유기치사', '존속학대', '주거수색', '주거침입',
       '주거침입교사', '주거침입미수', '주민등록법위반', '주민등록법위반교사', '주민등록법위반방조',
       '주식회사의외부감사에관한법률위반', '주차장법위반', '주차장법위반방조', '주택법위반', '준사기',
       '중과실장물취득', '중과실치사', '중과실치상', '중소기업제품구매촉진및판로지원에관한법률위반',
       '중소기업협동조합법위반', '중실화', '중증장애인생산품우선구매특별법위반', '증거위조', '증거은닉',
       '증거은닉교사', '증거인멸', '증거인멸교사', '증인도피', '증인은닉', '지방교육자치에관한법률위반',
       '지방세기본법위반', '지방세법위반', '지방재정법위반', '지하수법위반', '직권남용가혹행위', '직권남용감금',
       '직권남용권리행사방해', '직권남용체포', '직무유기', '직무유기(군)', '직업안정법위반',
       '집합건물의소유및관리에관한법률', '집합건물의소유및관리에관한법률위반', '집회및시위에관한법률위반',
       '채권의공정한추심에관한법률위반', '채권의공정한추심에관한법률위반교사', '채무자회생및파산에관한법률위반', '철도법위반',
       '철도안전법위반', '청소년기본법위반', '청소년보호법위반', '청소년의성보호에관한법률위반',
       '청소년의성보호에관한법률위반(강요행위등)', '청소년의성보호에관한법률위반(성매수)',
       '청소년의성보호에관한법률위반(청소년매매)', '청소년의성보호에관한법률위반(청소년이용음란물제작·배포등)',
       '체육시설의설치·이용에관한법률위반', '초·중등교육법위반', '초고층및지하연계복합건축물재난관리에관한특별법위반',
       '총포·도검·화약류등단속법위반', '총포·도검·화약류등의안전관리에관한법률위반', '추행', '축산물가공처리법위반',
       '축산물위생관리법위반', '축산법위반', '출입국관리법위반', '출판문화산업진흥법위반', '출판물에의한명예훼손',
       '치료감호법위반', '친환경농어업육성및유기식품등의관리·지원에관한법률위반', '컴퓨터등사용사기', '컴퓨터등사용사기미수',
       '컴퓨터등사용사기방조', '컴퓨터등손괴업무방해', '컴퓨터등장애업무방해', '컴퓨터등장애업무방해방조',
       '콘텐츠산업진흥법위반', '토양환경보전법위반', '통신비밀보호법위반', '통화변조', '통화위조', '퇴거불응',
       '특수강요', '특수건조물침입', '특수건조물침입교사', '특수건조물침입미수', '특수공무집행방해',
       '특수공무집행방해치상', '특수공용건조물파괴', '특수공용물건손상', '특수공용물건손상미수', '특수공용물건은닉',
       '특수방실침입', '특수주거침입', '특수주거침입교사', '특수주거침입미수', '특수퇴거불응',
       '특정강력범죄의처벌에관한특례법위반', '특정경제범죄가중처벌등에관한법률위반(배임)',
       '특정경제범죄가중처벌등에관한법률위반(배임)방조', '특정경제범죄가중처벌등에관한법률위반(사금융알선등)',
       '특정경제범죄가중처벌등에관한법률위반(사기)', '특정경제범죄가중처벌등에관한법률위반(사기)방조',
       '특정경제범죄가중처벌등에관한법률위반(수재등)', '특정경제범죄가중처벌등에관한법률위반(알선수재)',
       '특정경제범죄가중처벌등에관한법률위반(횡령)', '특정금융거래정보의보고및이용등에관한법률위반',
       '특정범죄가중처벌등에관한법률위반(감금)', '특정범죄가중처벌등에관한법률위반(관세)',
       '특정범죄가중처벌등에관한법률위반(뇌물)', '특정범죄가중처벌등에관한법률위반(도주치사)',
       '특정범죄가중처벌등에관한법률위반(도주치상)', '특정범죄가중처벌등에관한법률위반(도주치상)교사',
       '특정범죄가중처벌등에관한법률위반(독직폭행)', '특정범죄가중처벌등에관한법률위반(마약)',
       '특정범죄가중처벌등에관한법률위반(면담강요등)', '특정범죄가중처벌등에관한법률위반(알선수재)',
       '특정범죄가중처벌등에관한법률위반(위험운전치사)', '특정범죄가중처벌등에관한법률위반(위험운전치사상)',
       '특정범죄가중처벌등에관한법률위반(위험운전치상)', '특정범죄가중처벌등에관한법률위반(유기도주치사)',
       '특정범죄가중처벌등에관한법률위반(유기도주치상)', '특정범죄가중처벌등에관한법률위반(조세)',
       '특정범죄가중처벌등에관한법률위반(체포)', '특정범죄가중처벌등에관한법률위반(통화위조)',
       '특정범죄가중처벌등에관한법률위반(향정)', '특정범죄가중처벌등에관한법률위반(허위세금계산서교부등)',
       '특정범죄신고자등보호법위반', '특정범죄자에대한보호관찰및전자장치부착등에관한법률위반',
       '특정범죄자에대한위치추적전자장치부착등에관한법률위반', '특허법위반', '파견근로자보호등에관한법률위반',
       '편의시설부정이용', '편의시설부정이용미수', '편지개봉', '평생교육법', '폐기물관리법위반',
       '폭력행위등처벌에관한법률위반', '폭발물사용', '폭발성물건파열', '폭발성물건파열미수',
       '표시·광고의공정화에관한법률위반', '품질경영및공산품안전관리법위반', '풍속영업의규제에관한법률',
       '풍속영업의규제에관한법률위반', '피의사실공표', '하도급거래공정화에관한법률위반', '하수도법위반', '하천법위반',
       '학교보건법위반', '학교폭력예방및대책에관한법률위반', '학대', '학대치상',
       '학원의설립·운영및과외교습에관한법률위반', '학원의설립·운영에관한법률위반',
       '한강수계상수원수질개선및주민지원등에관한법률위반', '한국마사회법위반', '한국마사회법위반(도박개장등)',
       '한국마사회법위반(도박등)', '한국마사회법위반(도박등)방조', '한부모가족지원법위반', '할부거래에관한법률',
       '할부거래에관한법률위반', '항공법위반', '항공보안법위반', '항공안전법위반', '항만법위반', '해외건설촉진법위반',
       '해외이주법위반', '행정사법위반', '향토예비군설치법위반', '허가증불실기재', '허위감정', '허위공문서작성',
       '허위유가증권작성', '허위작성공문서행사', '허위진단서작성', '현존건조물방화', '현존건조물방화미수',
       '현존건조물방화예비', '현존건조물방화치사', '현존건조물방화치상', '현존자동차방화미수', '현존자동차방화예비',
       '현주건조물방화', '현주건조물방화미수', '현주건조물방화예비', '현주건조물방화치사', '현주건조물방화치상',
       '협동조합기본법위반', '형사사법절차전자화촉진법위반', '형사소송법위반', '형의실효등에관한법률위반',
       '형의집행및수용자의처우에관한법률위반', '화물자동차운수사업법위반', '화염병사용등의처벌에관한법률위반', '화장품법위반',
       '화재예방,소방시설설치·유지및안전관리에관한법률위반',
       '화학·생물무기의금지및특정화학물질·생물작용제등의제조·수출입규제등에관한법률위',
       '화학무기·생물무기의 금지와특정화학물질·생물작용제 등의 제조·수출입규제등에관한', '화학물질관리법위반',
       '화학물질관리법위반(환각물질흡입)', '화학물질의등록및평가등에관한법률위반', '환경기술및환경산업지원법위반',
       '환경분야시험·검사등에관한법률위반', '환경영향평가법위반', '환경오염피해배상책임및구제에관한법률위반', '횡령',
       '횡령미수', '후천성면역결핍증예방법위반']

---

# 3. 연도별, 관할서별 절도 폭력 검거수

## 3-1.절도, 폭력

In [42]:
#절도폭력
gr_절폭 = pd.DataFrame(df[df["crm"].isin(절폭_list)].groupby(["jur_stn","date"]).count()["crm"]).reset_index()
df_절폭 = gr_절폭.sort_values(["jur_stn","date"]).rename(columns = {"crm":"절도폭력_검거수"})
df_절폭

,jur_stn,date,절도폭력_검거수
0,경기수원남부경찰서,2017,4324
1,경기수원남부경찰서,2018,4141
2,경기수원남부경찰서,2019,4407
3,경기수원서부경찰서,2017,2726
4,경기수원서부경찰서,2018,2641
...,...,...,...
118,서울혜화경찰서,2018,1055
119,서울혜화경찰서,2019,1095
120,충남세종경찰서,2017,872
121,충남세종경찰서,2018,831


##  3-2. 강도, 살인

In [43]:
#강도살인
gr_강살 = pd.DataFrame(df[df["crm"].isin(강살_list)].groupby(["jur_stn","date"]).count()["crm"]).reset_index()
df_강살 = gr_강살.sort_values(["jur_stn","date"]).rename(columns = {"crm":"강도살인_검거수"})
df_강살

,jur_stn,date,강도살인_검거수
0,경기수원남부경찰서,2017,14
1,경기수원남부경찰서,2018,15
2,경기수원남부경찰서,2019,30
3,경기수원서부경찰서,2017,15
4,경기수원서부경찰서,2018,9
...,...,...,...
118,서울혜화경찰서,2018,3
119,서울혜화경찰서,2019,3
120,충남세종경찰서,2017,8
121,충남세종경찰서,2018,10


## 3-3. 교통사고

In [44]:
#교통사고
gr_교통 = pd.DataFrame(df[df["crm"].isin(교통_list)].groupby(["jur_stn","date"]).count()["crm"]).reset_index()
df_교통 = gr_교통.sort_values(["jur_stn","date"]).rename(columns = {"crm":"교통사고_검거수"})
df_교통

,jur_stn,date,교통사고_검거수
0,경기수원남부경찰서,2017,4779
1,경기수원남부경찰서,2018,5014
2,경기수원남부경찰서,2019,4272
3,경기수원서부경찰서,2017,1928
4,경기수원서부경찰서,2018,2103
...,...,...,...
118,서울혜화경찰서,2018,868
119,서울혜화경찰서,2019,744
120,충남세종경찰서,2017,742
121,충남세종경찰서,2018,1432


## 3-4. 법질서준수도( 기초질서, 집회지위질서 등 -> 5대범죄, 교통사고 제외)

In [45]:
#법질서준수도
gr_질서 = pd.DataFrame(df[df["crm"].isin(질서_list)].groupby(["jur_stn","date"]).count()["crm"]).reset_index()
df_질서 = gr_질서.sort_values(["jur_stn","date"]).rename(columns = {"crm":"법질서_검거수"})
df_질서

,jur_stn,date,법질서_검거수
0,경기수원남부경찰서,2017,7432
1,경기수원남부경찰서,2018,6193
2,경기수원남부경찰서,2019,7047
3,경기수원서부경찰서,2017,4852
4,경기수원서부경찰서,2018,3336
...,...,...,...
118,서울혜화경찰서,2018,1544
119,서울혜화경찰서,2019,1700
120,충남세종경찰서,2017,1869
121,충남세종경찰서,2018,1545


## 3-5. 전반적 안전도(ALL, 모든범죄건수)

In [46]:
#법질서준수도
gr_all = pd.DataFrame(df.groupby(["jur_stn","date"]).count()["crm"]).reset_index()
df_all = gr_all.sort_values(["jur_stn","date"]).rename(columns = {"crm":"전반적_검거수"})
df_all

,jur_stn,date,전반적_검거수
0,경기수원남부경찰서,2017,16775
1,경기수원남부경찰서,2018,15619
2,경기수원남부경찰서,2019,16021
3,경기수원서부경찰서,2017,9682
4,경기수원서부경찰서,2018,8220
...,...,...,...
118,서울혜화경찰서,2018,3529
119,서울혜화경찰서,2019,3596
120,충남세종경찰서,2017,3553
121,충남세종경찰서,2018,3897


# 4. 모든 파일 병합

In [47]:
df_범죄검거원표 = pd.concat([df_절폭, df_강살.iloc[:,-1] , df_교통.iloc[:,-1], df_질서.iloc[:,-1] , df_all.iloc[:,-1]], axis = 1)
df_범죄검거원표

,jur_stn,date,절도폭력_검거수,강도살인_검거수,교통사고_검거수,법질서_검거수,전반적_검거수
0,경기수원남부경찰서,2017,4324,14,4779,7432,16775
1,경기수원남부경찰서,2018,4141,15,5014,6193,15619
2,경기수원남부경찰서,2019,4407,30,4272,7047,16021
3,경기수원서부경찰서,2017,2726,15,1928,4852,9682
4,경기수원서부경찰서,2018,2641,9,2103,3336,8220
...,...,...,...,...,...,...,...
118,서울혜화경찰서,2018,1055,3,868,1544,3529
119,서울혜화경찰서,2019,1095,3,744,1700,3596
120,충남세종경찰서,2017,872,8,742,1869,3553
121,충남세종경찰서,2018,831,10,1432,1545,3897


# 5. 전처리완성본 파일저장

In [49]:
df_범죄검거원표.to_csv("../전처리완성본/7_범죄검거원표_전처리완성본")